In [ ]:
import rdflib

In [ ]:
def remove_prefix(s1:str, s2:str):
    min_len = min(len(s1), len(s2))
    for i in range(1, min_len):
        if s1[:i] != s2[:i]:
            s1 = s1[i-1:]
            s2 = s2[i-1:]
            break
    return [s1, s2]

In [ ]:
schema_file = open('\\\\Client\\G$\\EBO\\DSH\\Nijhuis\\nqm\\data\\raw\\schema.ttl', encoding = 'utf-8')
rdf_schema = rdflib.Graph()
rdf_schema.parse(file=schema_file, format="turtle")

linkbase_file = open('\\\\Client\\G$\\EBO\\DSH\\Nijhuis\\nqm\\data\\raw\\linkbase.ttl', encoding = 'utf-8')
rdf_linkbase = rdflib.Graph()
rdf_linkbase.parse(file=linkbase_file, format="turtle")

In [ ]:
combinations = []
for triple in rdf_linkbase.triples((None,None,rdflib.term.URIRef('https://w3id.org/vocab/xbrlllocator'))):
    found = False
    for triple2 in rdf_linkbase.triples((triple[0],rdflib.term.URIRef('https://w3id.org/vocab/xbrllconcept'),None)):
        combination = [triple[0], triple2[2]]
        combinations.append(combination)
        found = True
    if found:
        rdf_linkbase.remove((triple[0], None, None))        

In [ ]:
for combination in combinations:
    for triple in rdf_linkbase.triples((combination[0],None,None)):
        rdf_linkbase.add((combination[1], triple[1], triple[2])) 
    rdf_linkbase.remove((combination[0], None, None))       
    for triple in rdf_linkbase.triples((None, None, combination[0])):
        rdf_linkbase.add((triple[0], triple[1], combination[1])) 
    rdf_linkbase.remove((None, None, combination[0]))       
            

In [ ]:
combinations = []
j = 0
for triple in rdf_linkbase.triples((None,None,rdflib.term.URIRef('https://w3id.org/vocab/xbrllconcept-label'))):
    from_key = None
    values = []
    for triple2 in rdf_linkbase.triples((triple[0],rdflib.term.URIRef('http://www.xbrl.org/2003/XLinkfrom'),None)):
        from_key = remove_prefix(str(triple2[0]), str(triple2[2]))[1]
    for triple2 in rdf_linkbase.triples((triple[0],rdflib.term.URIRef('http://www.xbrl.org/2003/XLinkto'),None)):
        for triple3 in rdf_linkbase.triples((triple2[2], rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#value'), None)):
            values.append(triple3[2])
    if (from_key is not None) and (len(values) > 0):
        combinations.append([from_key, values, triple[0]])

In [ ]:
for triple in rdf_schema.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#id'), None)):
    for combination in combinations:
        if str(triple[2]) in combination:
            for string in combination[1]:
                rdf_schema.add((triple[0], rdflib.term.URIRef('https://w3id.org/vocab/xbrllconcept-label'), string))
        # rdf_linkbase.remove((trip[0], None, None))

In [ ]:
fg = open('..\\data\\raw\\linkbase2.rdf', "w", encoding = "utf-8")
fg.write(rdf_linkbase.serialize(format='turtle').decode('utf-8'))
fg.close()

fg = open('..\\data\\raw\\schema2.rdf', "w", encoding = "utf-8")
fg.write(rdf_schema.serialize(format='turtle').decode('utf-8'))
fg.close()